In [1]:
import os
import openai

In [2]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("sales_data_sample.csv")

In [5]:
import sqlalchemy

In [6]:
from sqlalchemy import create_engine
from sqlalchemy import text

In [7]:
temp_db = create_engine('sqlite:///:memory:', echo=True)

In [8]:
data = df.to_sql(name='Sales', con=temp_db)

2023-05-11 21:50:08,125 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-11 21:50:08,127 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Sales")
2023-05-11 21:50:08,127 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-11 21:50:08,128 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Sales")
2023-05-11 21:50:08,128 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-05-11 21:50:08,129 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Sales" (
	"index" BIGINT, 
	"ORDERNUMBER" BIGINT, 
	"QUANTITYORDERED" BIGINT, 
	"PRICEEACH" FLOAT, 
	"SALES" FLOAT, 
	"ORDERDATE" TEXT, 
	"QTR_ID" BIGINT, 
	"MONTH_ID" BIGINT, 
	"YEAR_ID" BIGINT, 
	"PRODUCTLINE" TEXT, 
	"PHONE" TEXT, 
	"ADDRESSLINE1" TEXT, 
	"CITY" TEXT, 
	"STATE" TEXT, 
	"POSTALCODE" TEXT, 
	"COUNTRY" TEXT, 
	"CONTACTLASTNAME" TEXT, 
	"CONTACTFIRSTNAME" TEXT
)


2023-05-11 21:50:08,129 INFO sqlalchemy.engine.Engine [no key 0.00029s] ()
2023-05-11 21:50:08,130 INFO sqlalchemy.engine.Engine CREATE INDEX "ix_Sales_index" ON "Sales" (

In [9]:
with temp_db.connect() as conn:
    result = conn.execute(text("SELECT SUM(SALES) FROM Sales"))

2023-05-11 21:50:08,169 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-05-11 21:50:08,170 INFO sqlalchemy.engine.Engine SELECT SUM(SALES) FROM Sales
2023-05-11 21:50:08,170 INFO sqlalchemy.engine.Engine [generated in 0.00111s] ()
2023-05-11 21:50:08,171 INFO sqlalchemy.engine.Engine ROLLBACK


In [10]:
result.all()

[(10032628.85000001,)]

In [11]:
def create_table_definition(df):
    prompt = """### sqlite SQL table, with its properties:
    #
    # Sales({})
    #
    """.format(",".join(str(col) for col in df.columns))

    return prompt

In [12]:
print(create_table_definition(df))

### sqlite SQL table, with its properties:
    #
    # Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,SALES,ORDERDATE,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,PHONE,ADDRESSLINE1,CITY,STATE,POSTALCODE,COUNTRY,CONTACTLASTNAME,CONTACTFIRSTNAME)
    #
    


In [13]:
def prompt_input():
    nlp_text = input("Enter the info you want: ")
    return nlp_text

In [16]:
def combine_prompts(df, query_prompt):
    definition = create_table_definition(df)
    query_init_string = f"### A query to answer: {query_prompt}\nSELECT"
    return definition + query_init_string

In [17]:
nlp_text = prompt_input()
combine_prompts(df, nlp_text)

'### sqlite SQL table, with its properties:\n    #\n    # Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,SALES,ORDERDATE,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,PHONE,ADDRESSLINE1,CITY,STATE,POSTALCODE,COUNTRY,CONTACTLASTNAME,CONTACTFIRSTNAME)\n    #\n    ### A Query to answer: total sales per quarter\nSELECT'